In [1]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation    
sns.set(color_codes=True)

pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
# Load the dataset
data_path = '/Users/kajalshukla/Desktop/MLOPs/Assignments/Assignment-1/adsp_ml_ops/data/athletes.csv'
data = pd.read_csv(data_path)

In [3]:
data.head(3)

,athlete_id,name,region,team,affiliate,gender,age,height,weight,fran,helen,grace,filthy50,fgonebad,run400,run5k,candj,snatch,deadlift,backsq,pullups,eat,train,background,experience,schedule,howlong
0,2554.0,Pj Ablang,South West,Double Edge,Double Edge CrossFit,Male,24.0,70.0,166.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,220.0,NaN,400.0,305.0,NaN,NaN,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|I r...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|
1,3517.0,Derek Abdella,NaN,NaN,NaN,Male,42.0,70.0,190.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I have a coach who determines my programming|I...,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|
2,4691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Raw data

## Data Cleaning

In [4]:
## Remove null values of relevant columns
relevant_columns = ['region', 'age', 'weight', 'height', 'howlong', 'gender', 'eat', 'train', 
                    'background', 'experience', 'schedule', 'deadlift', 'candj', 'snatch', 'backsq']
data = data.dropna(subset=relevant_columns)

In [5]:
## Remove irrelevant columns
irrelevant_columns = ['affiliate', 'team', 'name', 'athlete_id', 'fran', 'helen', 'grace',
                      'filthy50', 'fgonebad', 'run400', 'run5k', 'pullups', 'train']
data = data.drop(columns=irrelevant_columns)

In [6]:
data.head(5)

,region,gender,age,height,weight,candj,snatch,deadlift,backsq,eat,background,experience,schedule,howlong
6,South Central,Male,21.0,72.0,175.0,0.0,0.0,0.0,0.0,I eat quality foods but don't measure the amou...,I have no athletic background besides CrossFit|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 3+ times a wee...,2-4 years|
13,Central East,Male,43.0,71.0,185.0,0.0,0.0,0.0,0.0,I eat quality foods but don't measure the amount|,I have no athletic background besides CrossFit|,I began CrossFit with a coach (e.g. at an affi...,I usually only do 1 workout a day|I typically ...,2-4 years|
21,Southern California,Male,30.0,71.0,200.0,235.0,175.0,385.0,315.0,I eat whatever is convenient|,I played youth or high school level sports|I p...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 1x a week|I ty...,1-2 years|
22,Africa,Male,28.0,70.0,176.0,187.0,134.0,335.0,254.0,I eat 1-3 full cheat meals per week|,I have no athletic background besides CrossFit|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 1x a week|,2-4 years|
27,North East,Male,35.0,68.0,225.0,285.0,205.0,440.0,405.0,I eat quality foods but don't measure the amount|,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,I typically rest 4 or more days per month|,2-4 years|


In [7]:
# Clean survey data
decline_dict = {'Decline to answer|': np.nan}
data = data.replace(decline_dict)
data = data.dropna(subset=['background', 'experience', 'schedule', 'howlong', 'eat'])


### Determine the highest variability in columns to calculate total_lift

In [8]:
#Calculate standard deviation
numeric_columns = ['age', 'weight', 'height', 'howlong', 'deadlift', 'candj', 'snatch', 'backsq']

data_v1_std_devs = data[numeric_columns].std()

#Sort columns by their standard devation
sorted_columns_v1 = data_v1_std_devs.sort_values(ascending=False).index.tolist()

/var/folders/dn/9qflv0c57rl7ynhwt9tnqmrr0000gn/T/ipykernel_36804/2011793858.py:4: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_v1_std_devs = data[numeric_columns].std()


In [9]:
print(sorted_columns_v1)

['deadlift', 'height', 'backsq', 'snatch', 'candj', 'weight', 'age']


### The highest variability to calculate total_lift is 'deadlift', 'backsq', 'candj', 'snatch'

In [10]:
data['total_lift'] = data['deadlift'] + data['candj'] + data['snatch'] + data['backsq']


In [11]:
data = data.drop_duplicates()
print(data.isnull().sum())

region        0
gender        0
age           0
height        0
weight        0
candj         0
snatch        0
deadlift      0
backsq        0
eat           0
background    0
experience    0
schedule      0
howlong       0
total_lift    0
dtype: int64


## Dealing with Categorical Values

In [12]:
# One-Hot Encoding for categorical columns
categorical_cols = ['region', 'gender', 'eat', 'background', 'experience', 'schedule', 'howlong']
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

In [13]:
# Convert float columns to int
float_cols = ['age', 'height', 'weight', 'candj', 'snatch', 'deadlift', 'backsq', 'total_lift']
for col in float_cols:
    data[col] = data[col].astype(int)

In [14]:
# Verify the changes
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns

data.dtypes

age                                                                                                                                                                                                                                                                                                                           int64
height                                                                                                                                                                                                                                                                                                                        int64
weight                                                                                                                                                                                                                                                                                                                        int64
candj                       

## Save the raw data and push it using dvc tool

In [15]:
pip install dvc

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [16]:
!dvc init


Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [17]:
!git status

On branch monika_v2
Your branch is ahead of 'origin/monika_v2' by 1 commit.
  (use "git push" to publish your local commits)

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   .dvc/config
	modified:   data/athletes.csv.dvc

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .DS_Store
	modified:   data.ipynb
	modified:   data/.DS_Store
	deleted:    data/.gitignore
	deleted:    data/athletes.csv.dvc

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data/athletes.csv



In [18]:
# !git add .

In [19]:
# !git commit -m "Initialize DVC"

In [20]:
data.to_csv('data/athletes.csv', index=False)


In [21]:
!git rm --cached data/athletes.csv

fatal: pathspec 'data/athletes.csv' did not match any files


In [22]:
# !git commit -m "Stop tracking data/athletes.csv with Git"

In [23]:
# !dvc add data/athletes.csv


In [24]:
# !git add data/athletes.csv.dvc data/.gitignore


In [25]:
# !git commit -m "Track data/athletes.csv with DVC"

## Set up remote storage with DVC

In [26]:
!dvc remote add -d assign_1 gdrive://1W5BjxMSupxdh7RpcWGd-9HMDz8t8OcuH


Setting 'assign_1' as a default remote.


In [27]:
!dvc add data/athletes.csv

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/Users/kajalshukla/Desktop/MLOPs/Assignments/Assignment-
                                                                                
!
  0%|          |Adding data/athletes.csv to cache     0/? [00:00<?,     ?file/s]
  0%|          |Adding data/athletes.csv to cache     0/1 [00:00<?,     ?file/s]
  0%|          |Adding data/athletes.csv to cache     0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /Users/kajalshukla/Deskto0/? [00:00<?,    ?files/s]
  0%|          |Checking out /Users/kajalshukla/Deskto0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 15.03file/s]

To track the changes with git, run:

	g

In [28]:
!dvc commit data/athletes.csv.dvc

In [29]:
!dvc push


Everything is up to date.                                                       


In [30]:
!git add .dvc/config


In [31]:
!git add .

In [32]:
!git commit data/athletes.csv.dvc -m "adding version-1 data"

[monika_v2 80ffea9] adding version-1 data
 1 file changed, 2 insertions(+), 2 deletions(-)


In [33]:
!git push

Enumerating objects: 13, done.
Counting objects: 100% (13/13), done.
Delta compression using up to 8 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 1.07 KiB | 1.07 MiB/s, done.
Total 8 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (5/5), completed with 4 local objects.
To https://github.com/mariafshan/adsp_ml_ops.git
   d9e1d23..80ffea9  monika_v2 -> monika_v2


# Version - 2

## Remove Outliers

In [34]:
data_v2 = data

In [35]:
data_v2.head(2)

,age,height,weight,candj,snatch,deadlift,backsq,total_lift,region_Asia,region_Australia,region_Canada East,region_Canada West,region_Central East,region_Europe,region_Latin America,region_Mid Atlantic,region_North Central,region_North East,region_North West,region_Northern California,region_South Central,region_South East,region_South West,region_Southern California,gender_Female,gender_Male,eat_I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat quality foods but don't measure the amount|,eat_I eat quality foods but don't measure the amount|Decline to answer|,eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat strict Paleo|,eat_I eat strict Paleo|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat whatever is convenient|,eat_I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat whatever is convenient|,eat_I eat whatever is convenient|Decline to answer|,eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|,eat_I weigh and measure my food|Decline to answer|,eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|,eat_I weigh and measure my food|I eat strict Paleo|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|,eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat whatever is convenient|,eat_I weigh and measure my food|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,background_I have no athletic background besides Cro

In [36]:
# Remove outliers
data_v2 = data_v2[data_v2['weight'] < 1500]
# data = data[data['gender'] != '--']
data_v2 = data_v2[data_v2['age'] >= 18]
data_v2 = data_v2[(data_v2['height'] < 96) & (data_v2['height'] > 48)]

data_v2 = data_v2[
    (data_v2['deadlift'] > 0) & (data_v2['deadlift'] <= 1105) | 
    ((data_v2['gender_Female'] == 1) & (data_v2['deadlift'] <= 636))
]
data_v2 = data_v2[(data_v2['candj'] > 0) & (data_v2['candj'] <= 395)]
data_v2 = data_v2[(data_v2['snatch'] > 0) & (data_v2['snatch'] <= 496)]
data_v2 = data_v2[(data_v2['backsq'] > 0) & (data_v2['backsq'] <= 1069)]

In [37]:
data_v2.head(2)

,age,height,weight,candj,snatch,deadlift,backsq,total_lift,region_Asia,region_Australia,region_Canada East,region_Canada West,region_Central East,region_Europe,region_Latin America,region_Mid Atlantic,region_North Central,region_North East,region_North West,region_Northern California,region_South Central,region_South East,region_South West,region_Southern California,gender_Female,gender_Male,eat_I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat quality foods but don't measure the amount|,eat_I eat quality foods but don't measure the amount|Decline to answer|,eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat strict Paleo|,eat_I eat strict Paleo|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat whatever is convenient|,eat_I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat whatever is convenient|,eat_I eat whatever is convenient|Decline to answer|,eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|,eat_I weigh and measure my food|Decline to answer|,eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|,eat_I weigh and measure my food|I eat strict Paleo|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|,eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat whatever is convenient|,eat_I weigh and measure my food|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,background_I have no athletic background besides Cro

## Feature Engineering

### Body Mass Index

In [38]:
data_v2['bmi'] = data_v2['weight'] / (data_v2['height'] / 100)**2

### Dropping eat column

### Binning weight columns

In [39]:
# Sample bins and labels
bins_weight = [0, 125, 165, 205, 245, float('inf')]
labels_weight = ['Underweight', 'Normal weight', 'Overweight', 'Obese', 'Extremely Obese']


In [40]:
data_v2['weight_group'] = pd.cut(data_v2['weight'], bins=bins_weight, labels=labels_weight, right=False)


### Binning Age 

In [41]:
# Binning the age
bins_age = [0, 18, 35, 60, float('inf')]
labels_age = ['Teen', 'Young Adult', 'Adult', 'Senior']
data_v2['age_group'] = pd.cut(data_v2['age'], bins=bins_age, labels=labels_age, right=False)


In [42]:
data_v2.head(3)

,age,height,weight,candj,snatch,deadlift,backsq,total_lift,region_Asia,region_Australia,region_Canada East,region_Canada West,region_Central East,region_Europe,region_Latin America,region_Mid Atlantic,region_North Central,region_North East,region_North West,region_Northern California,region_South Central,region_South East,region_South West,region_Southern California,gender_Female,gender_Male,eat_I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat quality foods but don't measure the amount|,eat_I eat quality foods but don't measure the amount|Decline to answer|,eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat strict Paleo|,eat_I eat strict Paleo|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat whatever is convenient|,eat_I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat whatever is convenient|,eat_I eat whatever is convenient|Decline to answer|,eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|,eat_I weigh and measure my food|Decline to answer|,eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|,eat_I weigh and measure my food|I eat strict Paleo|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|,eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat whatever is convenient|,eat_I weigh and measure my food|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,background_I have no athletic background besides Cro

In [43]:
data_v2.dtypes

age                                                                                                                                                                                                                                                                                                                              int64
height                                                                                                                                                                                                                                                                                                                           int64
weight                                                                                                                                                                                                                                                                                                                           int64
candj              

In [44]:
# One-Hot Encoding for categorical columns
categorical_cols = ['age_group', 'weight_group']
data_v2 = pd.get_dummies(data_v2, columns=categorical_cols, drop_first=True)

In [45]:
# Convert float columns to int
data_v2['bmi'] = data_v2['bmi'].astype(int)

In [46]:
data_v2.dtypes

age                                                                                                                                                                                                                                                                                                                           int64
height                                                                                                                                                                                                                                                                                                                        int64
weight                                                                                                                                                                                                                                                                                                                        int64
candj                       

## Save version 2

In [47]:
data_v2.to_csv('data/athletes.csv', index=False)

In [48]:
!dvc add data/athletes.csv

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/Users/kajalshukla/Desktop/MLOPs/Assignments/Assignment-
                                                                                
!
  0%|          |Adding data/athletes.csv to cache     0/? [00:00<?,     ?file/s]
  0%|          |Adding data/athletes.csv to cache     0/1 [00:00<?,     ?file/s]
  0%|          |Adding data/athletes.csv to cache     0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /Users/kajalshukla/Deskto0/? [00:00<?,    ?files/s]
  0%|          |Checking out /Users/kajalshukla/Deskto0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 51.58file/s]

To track the changes with git, run:

	g

In [49]:
!dvc commit data/athletes.csv.dvc

In [50]:
!dvc push

Everything is up to date.                                                       


In [51]:
!git add .

In [52]:
!git commit data/athletes.csv.dvc -m "Updated data to version 2"


[monika_v2 34bd5a2] Updated data to version 2
 1 file changed, 2 insertions(+), 2 deletions(-)


In [53]:
!git push

Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 427 bytes | 427.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/mariafshan/adsp_ml_ops.git
   80ffea9..34bd5a2  monika_v2 -> monika_v2
